In [ ]:
import flask
from flask import request, jsonify
import joblib
import pandas as pd
import json
import dill
import lime
import lime.lime_tabular
import pickle
import numpy as np

app = flask.Flask(__name__)
app.config["DEBUG"] = False

#Chargement du tableau et du modèle
with open('df_test.pkl', 'rb') as f:
    df = pickle.load(f)

feats = [f for f in df.columns if f not in [
    'TARGET', 'SK_ID_CURR', 'SK_ID_BUREAU', 'SK_ID_PREV', 'index']]    

df.drop(columns=["index"], inplace=True)
df.set_index("SK_ID_CURR", inplace=True)


with open('lgbm_model.pkl', 'rb') as f:
    model = pickle.load(f)
    print("modéle load ok")
    
with open('pred_df.pkl', 'rb') as f:
    pred_df = pickle.load(f)    


def make_prediction(client_id):
    return model.predict_proba([df[feats].loc[client_id]])[0, 1]
    
    
@app.route('/', methods=['GET'])
def index():
    return {'message': 'Bonjour,tout le monde'}


@app.route('/score_min/', methods=['GET'])
def score_min():
    return {"score_min" : 0.55} 


@app.route('/predict', methods=["GET"])
def proba():
    if 'client_id' in request.args:
        client_id = int(request.args["client_id"])
        pred = make_prediction(client_id)
        print(pred)
        return {"proba" : pred}
    else:
        return "Error"
"""
@app.route('/predict', methods=["GET"])
def proba():
    if 'client_id' in request.args:
        client_id = int(request.args["client_id"])

        # Vérifier si le client existe déjà dans le DataFrame
        if client_id in pred_df.index:
            # Si le client est déjà présent, renvoyer directement la valeur de sa colonne "predict"
            pred = pred_df.loc[client_id]
        else:
            # Si le client n'est pas trouvé, effectuer la prédiction et l'enregistrer dans le DataFrame
            pred = make_prediction(client_id)
            pred_df.loc[client_id, "predict"] = pred

        return {"proba" : pred}
    else:
        return "Error"
@app.route('/feats/', methods=["GET"])
def feats_ret():
    return feats
#json.dumps(feats) 
#.tolist()
@app.route('/importances', methods=["GET"])
def importances():
    if 'client_id' in request.args:
        client_id = int(request.args["client_id"])
        shap_vals = explain(client_id).tolist()
        return json.dumps(shap_vals)
    else:
        return "Error"    

@app.route('/bar', methods=["GET"])
def bar():
    if 'client_id' in request.args:
        client_id = int(request.args["client_id"])
        feat = str(request.args["feature"])
        
        dff = df[feat]
        retour = []
        retour.append(float(dff.loc[client_id]))
        retour.append(np.mean(dff))
        del dff      
     
        return json.dumps(retour)
    else:
        return "Error"     
@app.route('/boxplot', methods=["GET"])
def boxplot():
    if 'feature' in request.args:
        feat = str(request.args["feature"])
        
        dff = df[feat]
     
        return json.dumps(dff.tolist())
    else:
        return "Error"   
 
if __name__ == "__main__":
    app.run(port=80)

modéle load ok
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:80
Press CTRL+C to quit
